# <div align="center"> 图像分类 </div>

In [1]:
from IPython.display import Markdown as md
import requests
import json
import consul
import time
import socket

## 1 全局配置

### 1.1 函数

In [2]:
# 获取本机器IP
def get_host_ip():
    ip=''
    try:
        s = socket.socket(socket.AF_INET,socket.SOCK_DGRAM)
        s.connect(('8.8.8.8',80))
        ip = s.getsockname()[0]
    finally:
        s.close()
    return ip

def delete_datacenter_key(key):
    try:
        client = consul.Consul(consul_addr, consul_port)
        client.kv.delete(key)
    except:
        pass

# API异步接口将过程中产出的数据同步到consul数据中心, 调用该接口可从中心获取数据 (数据中心仅供开发使用)
def query_async_data(key, trycnt=3):
    i = 0
    while i < trycnt:
        try:
            client = consul.Consul(consul_addr, consul_port)
            _, data = client.kv.get(key)
            return str(data['Value'], encoding="utf-8")
        except:
            i = i + 1
            time.sleep(2)
    return 'Query nothing!!!'
    
# 格式化打印json字符串
def print_json(text):
    if isinstance(text, str):
        print(json.dumps(json.loads(text), indent=4))
    else:
        print(json.dumps(text, indent=4))

## 1.2 变量

In [3]:
host = get_host_ip()
port = 8119

user = 'test'
uuid = 'abcdef'

consul_addr = !echo $(curl -s http://txt.go.sohu.com/ip/soip)| grep -P -o -i "(\d+\.\d+.\d+.\d+)"
consul_port = 8500
consul_addr = consul_addr[0]

query_trycnt = 25
max_epoch = 2

## 1.3 配置模板

In [4]:
cls_template_config = '''{
    "dataset": "cifar10", 
    "task": "cls", 
    "method": "image_classifier", 
    "data": {
        "num_classes": 10, 
        "data_dir": "/data/datasets/cv/cifar10", 
        "image_tool": "pil", 
        "input_mode": "RGB", 
        "workers": 1, 
        "mean_value": [
            124, 
            116, 
            104
        ], 
        "normalize": {
            "div_value": 255, 
            "mean": [
                0.485, 
                0.456, 
                0.406
            ], 
            "std": [
                0.229, 
                0.224, 
                0.225
            ]
        }
    }, 
    "train": {
        "batch_size": 32, 
        "aug_trans": {
            "trans_seq": [
                "random_hflip", 
                "random_border", 
                "random_crop"
            ], 
            "random_hflip": {
                "ratio": 0.5, 
                "swap_pair": [ ]
            }, 
            "random_border": {
                "ratio": 1, 
                "pad": [
                    4, 
                    4, 
                    4, 
                    4
                ], 
                "allow_outside_center": false
            }, 
            "random_crop": {
                "ratio": 1, 
                "crop_size": [
                    32, 
                    32
                ], 
                "method": "random", 
                "allow_outside_center": false
            }
        }, 
        "data_transformer": {
            "size_mode": "fix_size", 
            "input_size": [
                32, 
                32
            ], 
            "align_method": "only_pad"
        }
    }, 
    "val": {
        "batch_size": 32, 
        "aug_trans": {
            "trans_seq": [ ]
        }, 
        "data_transformer": {
            "size_mode": "fix_size", 
            "input_size": [
                32, 
                32
            ], 
            "align_method": "only_pad"
        }
    }, 
    "test": { 
    }, 
    "details": {
        "name_seq": [
            "plane", 
            "car", 
            "bird", 
            "cat", 
            "deer", 
            "dog", 
            "frog", 
            "horse", 
            "ship", 
            "truck"
        ], 
        "color_list": [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], [170, 255, 0],
                     [85, 255, 0], [0, 255, 0], [0, 255, 85], [0, 255, 170], [0, 255, 255],
                     [0, 170, 255], [0, 85, 255], [0, 0, 255], [85, 0, 255], [170, 0, 255],
                     [255, 0, 255], [255, 0, 170], [255, 0, 85], [255, 0, 170], [255, 0, 85]]
    }, 
    "network": {
        "model_name": "cls_model", 
        "backbone": "vgg19", 
        "pooled_size": [
            1, 
            1
        ]
    }, 
    "solver": {
        "lr": {
            "metric": "epoch", 
            "base_lr": 0.1, 
            "lr_policy": "multistep", 
            "step": {
                "gamma": 0.1, 
                "step_size": 30
            }, 
            "multistep": {
                "gamma": 0.1, 
                "stepvalue": [
                    150, 
                    250, 
                    350
                ]
            }
        }, 
        "optim": {
            "optim_method": "sgd", 
            "adam": {
                "betas": [
                    0.9, 
                    0.999
                ], 
                "eps": 1e-8, 
                "weight_decay": 0.0001
            }, 
            "sgd": {
                "weight_decay": 0.00004, 
                "momentum": 0.9, 
                "nesterov": false
            }
        }, 
        "display_iter": 20, 
        "save_iters": 2000, 
        "test_interval": 100, 
        "max_epoch": 360
    }, 
    "loss": {
        "loss_type": "ce_loss", 
        "loss_weights": {
            "ce_loss": {
                "ce_loss": 1
            }, 
            "soft_ce_loss": {
                "soft_ce_loss": 1
            }, 
            "mixup_ce_loss": {
                "mixup_ce_loss": 1
            }, 
            "mixup_soft_ce_loss": {
                "mixup_soft_ce_loss": 1
            }
        }, 
        "params": {
            "ce_loss": {
                "reduction": "mean", 
                "ignore_index": -1
            }, 
            "soft_ce_loss": {
                "reduction": "batchmean", 
                "label_smooth": 0.1
            }, 
            "mixup_ce_loss": {
                "reduction": "mean", 
                "ignore_index": -1
            }, 
            "mixup_soft_ce_loss": {
                "reduction": "batchmean", 
                "label_smooth": 0.1
            }
        }
    }
}'''

cls_template_config_dict = json.loads(cls_template_config)

## 2 CIFAR10

### 2.1 训练

In [5]:
cls_config = cls_template_config_dict

cls_config['dataset'] = 'cifar10'
cls_config['data']['data_dir'] = '/data/datasets/cifar10'
cls_config['train']['batch_size'] = 32
cls_config['details']['name_seq'] = [
    "plane", 
    "car", 
    "bird", 
    "cat", 
    "deer", 
    "dog", 
    "frog", 
    "horse", 
    "ship", 
    "truck"
]
cls_config['solver']['max_epoch'] = max_epoch
cls_config['solver']['optim']['optim_method'] = 'sgd'
cls_config['loss']['loss_type'] = 'ce_loss'
cls_config['network']['backbone'] = 'vgg19'
conf = json.dumps(cls_config)

### train.stop

In [14]:
op = 'train.stop'
uuid = 'cls_vgg19_cifar10'
data = '''{
    "op":"%s",
    "user": "%s",
    "service_name": "k12cv",
    "service_uuid": "%s",
    "service_params": {}
}''' % (op, user, uuid)

api = 'http://%s:%d/k12ai/framework/train' % (host, port)
print_json(requests.post(url=api, json=json.loads(data)).text)

{
    "code": 100200,
    "descr": "task service success"
}


### train.start

In [16]:
op = 'train.start'
uuid = 'cls_vgg19_cifar10'
data = '''{
    "op":"%s",
    "user": "%s",
    "service_name": "k12cv",
    "service_uuid": "%s",
    "service_params": %s
}''' % (op, user, uuid, conf)

api = 'http://%s:%d/k12ai/framework/train' % (host, port)
print_json(requests.post(url=api, json=json.loads(data)).text)

key = 'framework/%s/%s/%s/metrics'%(user, uuid, op)
delete_datacenter_key(key)

{
    "code": 100200,
    "descr": "task service success"
}


异步输出样例:

```json
{
    "version": "0.1.0",
    "type": "metrics",
    "tag": "framework",
    "op": "train.start",
    "user": "test",
    "service_uuid": "abcdef",
    "timestamp": 1574939332062,
    "datetime": "2019-11-28 19:08:52",
    "metrics": {
        "training_epochs": 1,
        "training_loss": 2.3058,
        "training_speed": 0.105,
        "lr": [
            0.1,
            0.1
        ],
        "validation_loss": 2.3041,
        "validation_accuracy": 0.10519999710842967,
        "validation_accuracy3": 0.3018999940156937,
        "validation_accuracy5": 0.49709998607635497
    }
}
```

In [ ]:
print(query_async_data(key, query_trycnt))

### 2.2 评估

In [ ]:
cls_config = cls_template_config_dict

cls_config['dataset'] = 'cifar10'
cls_config['data']['data_dir'] = '/data/datasets/cifar10'
cls_config['train']['batch_size'] = 32
cls_config['details']['name_seq'] = [
    "plane", 
    "car", 
    "bird", 
    "cat", 
    "deer", 
    "dog", 
    "frog", 
    "horse", 
    "ship", 
    "truck"
]
cls_config['solver']['max_epoch'] = max_epoch
cls_config['solver']['optim']['optim_method'] = 'sgd'
cls_config['loss']['loss_type'] = 'ce_loss'
cls_config['network']['backbone'] = 'vgg19'
conf = json.dumps(cls_config)

### evaluate.stop

In [ ]:
op = "evaluate.stop"
uuid = 'cls_vgg19_cifar10'
data = '''{
    "op":"%s", 
    "user": "%s",
    "service_name": "k12cv",
    "service_uuid": "%s",
    "service_params": {}
}''' % (op, user, uuid)

api = 'http://%s:%d/k12ai/framework/evaluate' % (host, port)
print_json(requests.post(url=api, json=json.loads(data)).text)

### evaluate.start

In [ ]:
op = "evaluate.start"
uuid = 'cls_vgg19_cifar10'
data = '''{
    "op":"%s", 
    "user": "%s",
    "service_name": "k12cv",
    "service_uuid": "%s",
    "service_params": %s
}''' % (op, user, uuid, conf)

api = 'http://%s:%d/k12ai/framework/evaluate' % (host, port)
print_json(requests.post(url=api, json=json.loads(data)).text)

key = 'framework/%s/%s/%s/metrics'%(user, uuid, op)
delete_datacenter_key(key)

In [ ]:
print(query_async_data(key, query_trycnt))

## 3 MNIST

### 3.1 训练

In [ ]:
cls_config = cls_template_config_dict

cls_config['dataset'] = 'mnist'
cls_config['data']['data_dir'] = '/data/datasets/cv/mnist'
cls_config['train']['batch_size'] = 32
cls_config['details']['name_seq'] = [
    "0", 
    "1", 
    "2", 
    "3", 
    "4", 
    "5", 
    "6", 
    "7", 
    "8", 
    "9"
]
cls_config['solver']['max_epoch'] = max_epoch
cls_config['solver']['optim']['optim_method'] = 'adam'
cls_config['loss']['loss_type'] = 'soft_ce_loss'
cls_config['network']['backbone'] = 'resnet18'
conf = json.dumps(cls_config)

### train.stop

In [ ]:
op = 'train.stop'
uuid = 'cls_resnet18_mnist'
data = '''{
    "op":"train.stop",
    "user": "%s",
    "service_name": "k12cv",
    "service_uuid": "%s",
    "service_params": {}
}''' % (user, uuid)

api = 'http://%s:%d/k12ai/framework/train' % (host, port)
print_json(requests.post(url=api, json=json.loads(data)).text)

### train.sart

In [ ]:
op = "train.start"
uuid = 'cls_resnet18_mnist'
data = '''{
    "op":"%s",
    "user": "%s",
    "service_name": "k12cv",
    "service_uuid": "%s",
    "service_params": %s
}''' % (op, user, uuid, conf)

api = 'http://%s:%d/k12ai/framework/train' % (host, port)
print_json(requests.post(url=api, json=json.loads(data)).text)

key = 'framework/%s/%s/%s/metrics'%(user, uuid, op)
delete_datacenter_key(key)

In [ ]:
print(query_async_data(key, query_trycnt))

### evaluate.stop

In [ ]:
op = "evaluate.stop"
uuid = 'cls_resnet18_mnist'
data = '''{
    "op":"%s", 
    "user": "%s",
    "service_name": "k12cv",
    "service_uuid": "%s",
    "service_params": {}
}''' % (op, user, uuid)

api = 'http://%s:%d/k12ai/framework/evaluate' % (host, port)
print_json(requests.post(url=api, json=json.loads(data)).text)

### evaluate.start

In [ ]:
op = "evaluate.start"
uuid = 'cls_resnet18_mnist'
data = '''{
    "op":"%s", 
    "user": "%s",
    "service_name": "k12cv",
    "service_uuid": "%s",
    "service_params": %s
}''' % (op, user, uuid, conf)

api = 'http://%s:%d/k12ai/framework/evaluate' % (host, port)
print_json(requests.post(url=api, json=json.loads(data)).text)

key = 'framework/%s/%s/%s/metrics'%(user, uuid, op)
delete_datacenter_key(key)

In [ ]:
print(query_async_data(key, query_trycnt))